In [12]:
#pip install youtube-transcript-api
from youtube_transcript_api import YouTubeTranscriptApi
import re

In [30]:
def get_youtube_transcript(video_url, chunk_size=50):
    video_id = re.findall(r"(?:v=|be/)([A-Za-z0-9_-]+)", video_url)[0]
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
    steps = []
    current = ""
    for entry in transcript:
        current += entry['text'] + " "
        if len(current.split()) >= chunk_size:
            steps.append(current.strip())
            current = ""
    if current:
        steps.append(current.strip())
    return steps

if __name__ == "__main__":
    url = "https://youtu.be/IK62Rk47aas"
    steps = get_youtube_transcript(url, chunk_size=40)
    for i, s in enumerate(steps, 1):
        print(f"\n--- Step {i} ---\n{s}\n")



--- Step 1 ---
Today we're going to show you how to automate the process of creating interviews, conducting resume based screening, and even executing first round video interviews using AI. The first step is to create an interview. This can be done in two easy


--- Step 2 ---
steps. First, provide the job description. Once you've done that, the AI will suggest questions based on the job description. You can customize these questions and then establish the interview. For users of the advanced plan, there's an extra step where you


--- Step 3 ---
can select an AI avatar. This ensures that the lip syncing aligns with the script. Let's start with the first step. If you already have a job description, enter it here. If you don't have a description, but you do know the


--- Step 4 ---
job title, such as looking for a 2-year experienced JavaScript developer with AWS experience, you can use the enhanced JD feature. The AI will generate a complete job description for you. If you want, you c

In [31]:
import openai  ## pip install openai
from tqdm import tqdm

In [75]:
openai.api_key = "YOUR_API_KEY"

def generate_test_cases_from_chunks(chunks):
    all_results = []
    for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
        prompt = f"""
        You are QAgenie — an AI QA assistant.
        
        Below is a chunk of a video transcript explaining how Recruter.ai works. Based on this, generate frontend test cases in Markdown and JSON formats.
        
        Focus on:
        - Core functionality
        - Edge cases
        - Accessibility
        - Cross-browser/mobile
        - Network failure handling
        
        Transcript Chunk:
        \"\"\"
        {chunk}
        \"\"\"
        
        Now generate:
        
        1.  Markdown test cases (human readable)
        2.  JSON format test cases (machine readable, for Playwright generation)
        
        Output format:
        
        ###  Human-Readable Test Cases (Markdown)
        - [TC-001] <Test case title>: Description, steps, expected result.
        - [TC-002] ...
        
        ---
        ### 🧩 JSON Test Cases
        [
          {{
            "id": "TC-001",
            "title": "Signup with valid data",
            "steps": ["Go to homepage", "Click 'Sign up'", "Enter valid email/password", "Click 'Submit'"],
            "expected": "User is signed up successfully",
            "tags": ["core", "signup"]
          }},
          ...
        ]
        """
        
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a QA assistant for frontend automation."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
        
        result = response.choices[0].message.content
        all_results.append(result)
    
    return all_results


In [76]:
import json

In [100]:
def generate_playwright_test(tc):
    title = tc["title"]
    tc_id = tc["id"]
    steps = tc["steps"]
    expected = tc["expected"]

    code_lines = [f"test('{tc_id}: {title}', async ({{ page }}) => {{"]

    for step in steps:
        step_lower = step.lower()
        if "go to" in step_lower:
            code_lines.append(f"  await page.goto('https://recruter.ai');")
        elif "click" in step_lower:
            text = step.split("Click")[-1].strip()
            code_lines.append(f"  await page.click('text={text}');")
        elif "enter valid data" in step_lower or "enter" in step_lower:
            code_lines.append(f"  await page.fill('input[type=\"email\"]', 'test@example.com');")
            code_lines.append(f"  await page.fill('input[type=\"password\"]', 'Test@123');")
        elif "submit" in step_lower:
            code_lines.append(f"  await page.click('text=Submit');")

    # Expected output handling (simple)
    if "dashboard" in expected.lower():
        code_lines.append(f"  await expect(page).toHaveURL(/.*dashboard/);")
    elif "error" in expected.lower():
        code_lines.append(f"  await expect(page.locator('.error')).toBeVisible();")

    code_lines.append("});\n")

    return "\n".join(code_lines)


In [101]:
# test_case_generator.py
from pathlib import Path

In [105]:
# Load test cases from JSON
with open("C:\\Users\\gauta\\Desktop\\Project\\qa-agent\\Test_case.JSON", "r") as f:
    test_cases = json.load(f)

In [121]:
# Generate test file
with open("signup.spec.ts", "w") as outfile:
    outfile.write("import { test, expect } from '@playwright/test';\n\n")
    for tc in test_cases:
        ts_code = generate_playwright_test(tc)
        outfile.write(ts_code)
